# Clustering Measurements

{{ Triumvirate }} provides algorithms for computing clustering statistics in
both Fourier and configuration space and in both local and global
plane-parallel approximations (see ['Background'](../background.rst) for
details).

The usage of these measurement algorithms is all very similar, so as
an example we will consider the bispectrum measurement below, and briefly
mention the differences for other measurements.

In [1]:
from triumvirate.threept import compute_bispec

## Ingredients

There are a number of inputs for measurements:
- catalogue objects (as {py:class}`~triumvirate.catalogue.ParticleCatalogue`;
  see the ['Particle Catalogue'](./Catalogue.ipynb) tutorial);
- measurement parameters (as {py:class}`~triumvirate.parameters.ParameterSet`
  or passed/overriden by keyword arguments; see the
  ['Parameter Set'](./Parameters.ipynb) tutorial);
- optional logger (as {py:class}`logging.Logger`; see the
  ['Customised Logger'](./Logger.ipynb) tutorial).

We will reuse `trv_logger`, `parameter_set`, `binning` and `catalogue` created
in tutorials ['Customised Logger'](./Logger.ipynb),
['Parameter Set'](./Parameters.ipynb), ['Binnig Scheme'](./Binning.ipynb) and
['Particle Catalogue'](./Catalogue.ipynb) as inputs.

In [2]:
from triumvirate.logger import setup_logger
from triumvirate.parameters import ParameterSet
from triumvirate.dataobjs import Binning

# Demo logger
trv_logger = setup_logger()

# Demo parameter set
try:
    parameter_set = ParameterSet(param_filepath="parameter_template.yml")
except OSError:
    from triumvirate.parameters import fetch_paramset_template

    parameter_dict = fetch_paramset_template('dict')

    for ax_name in ['x', 'y', 'z']:
        parameter_dict['boxsize'][ax_name] = 1000.
        parameter_dict['ngrid'][ax_name] = 128

    parameter_dict.update({
        'catalogue_type': 'sim',
        'statistic_type': 'bispec',
        'degrees'       : {'ell1': 0, 'ell2': 0, 'ELL': 0},
        'range'         : [0.005, 0.105],
        'num_bins'      : 10,
    })

    parameter_set = ParameterSet(param_dict=parameter_dict)

# Demo binning
binning = Binning('fourier', 'lin', bin_min=0.005, bin_max=0.105, num_bins=10)

[2023-02-23 23:44:39 (+00:00:00) STAT C++] Parameters validated.


In addition, we have used ``nbodykit`` to produce three types of
mock catalogues:

- The first is a simulation-like log-normal catalogue `catalogue_sim`
  in a cubic box of size $L = 1000.\,h^{-1}\,\mathrm{Mpc}$ with number density
  $\bar{n} = 5 \times 10^{-4} \,h^3\,\mathrm{Mpc}^{-3}$. The input cosmological
  parameters are $h = 0.6736, \Omega_{\mathrm{CDM},0} = 0.2645, 
  \Omega_{\mathrm{b},0} = 0.04930, A_s = 2.083 \times 10^{-9}$ and
  $n_s = 0.9649$, and the linear power spectrum at redshift $z = 1.$ with
  linear tracer bias $b_1 = 2$ is used.

- The second is a survey-like catalogue `catalogue_survey` based on
  the simulation-like one, with the catalogue cut to the inscribing sphere of
  radius $L/2$ inside the cubic box.

- The third is a uniform random catalogue `catalogue_rand` with
  number density $5 \bar{n}$ in the same spherical volume as
  the survey-like one.

In [3]:
import numpy as np
from nbodykit.cosmology import Cosmology

# Cosmology, matter power spectrum and bias at given redshift
cosmo = Cosmology(
    h=0.6736, Omega0_b=0.04930, Omega0_cdm=0.2645, A_s=2.083e-09, n_s=0.9649
)
redshift = 1.
bias = 2.

# Catalogue properties
density = 5.e-4
boxsize = 1000.

# Catalogue selectors
def cut_to_sphere(coords, boxsize):
    return np.less_equal(np.sqrt(np.sum(coords**2, axis=-1)), boxsize/2.)

In [4]:
# Create simulation-like catalogue, or load if existing.
catalogue_sim_filepath = "mock_catalogue_sim.dat"

try:
    catalogue_sim = np.loadtxt(
        catalogue_sim_filepath,
        dtype=[(axis, np.float64) for axis in ['x', 'y', 'z']]
    )
except FileNotFoundError:
    from nbodykit.lab import LinearPower, LogNormalCatalog
    powspec = LinearPower(cosmo, redshift)
    catalogue_sim = LogNormalCatalog(
        powspec, density, boxsize, bias=bias, Nmesh=256, seed=42
    )
    catalogue_sim['Position'] -= boxsize/2.
    np.savetxt(catalogue_sim_filepath, catalogue_sim['Position'].compute())

In [5]:
# Create survey-like catalogue.
try:
    catalogue_survey = catalogue_sim[
        cut_to_sphere(catalogue_sim['Position'], boxsize).compute()
    ]
except (IndexError, ValueError):
    catalogue_survey = catalogue_sim[
        cut_to_sphere(
            catalogue_sim[['x', 'y', 'z']]
            .view(np.float64).reshape(len(catalogue_sim), 3),
            boxsize
        )
    ]

In [6]:
# Create random catalogue, or load if existing.
catalogue_rand_filepath = "mock_catalogue_rand.dat"

try:
    catalogue_rand = np.loadtxt(
        catalogue_rand_filepath,
        dtype=[(axis, np.float64) for axis in ['x', 'y', 'z']]
    )
except FileNotFoundError:
    from nbodykit.lab import UniformCatalog
    catalogue_rand = UniformCatalog(5*density, boxsize, seed=42)
    catalogue_rand['Position'] -= boxsize/2.
    catalogue_rand = catalogue_rand[
        cut_to_sphere(catalogue_rand['Position'], boxsize).compute()
    ]
    np.savetxt(catalogue_rand_filepath, catalogue_rand['Position'].compute())

In [7]:
import warnings
from triumvirate.catalogue import ParticleCatalogue

warnings.filterwarnings('ignore', message=".*'nz' field.*")

catalogue_sim = ParticleCatalogue(
    *[catalogue_sim[coord_axis] for coord_axis in ['x', 'y', 'z']]
)
catalogue_survey = ParticleCatalogue(
    *[catalogue_survey[coord_axis] for coord_axis in ['x', 'y', 'z']],
    nz=density
)
catalogue_rand = ParticleCatalogue(
    *[catalogue_rand[coord_axis] for coord_axis in ['x', 'y', 'z']],
    nz=density
)

[2023-02-23 23:45:07 (+00:00:27) INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-02-23 23:45:07 (+00:00:27) INFO] NumExpr defaulting to 8 threads.


## Measurements

Having specified all the inputs, measurements can be made by simply passing
them as arguments to the relevant function:

In [8]:
results = compute_bispec(
    catalogue_survey, catalogue_rand,
    paramset=parameter_set,
    logger=trv_logger
)

[2023-02-23 23:45:09 (+00:00:29) INFO] Parameter set have been initialised.
[2023-02-23 23:45:09 (+00:00:29) STAT C++] Parameters validated.
[2023-02-23 23:45:09 (+00:00:29) INFO] Binning has been initialised.
[2023-02-23 23:45:09 (+00:00:29) INFO] Lines of sight have been initialised.
[2023-02-23 23:45:10 (+00:00:30) INFO] Catalogues have been aligned.
[2023-02-23 23:45:10 (+00:00:30) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-23 23:45:10 (+00:00:30) INFO C++] Catalogue loaded: 259444 particles with total sample weights 259444.000 (source=extdata).
[2023-02-23 23:45:10 (+00:00:31) INFO C++] Extents of particle coordinates: {'x': (2.438, 998.884), 'y': (0.879, 998.351), 'z': (0.364, 998.843)} (source=extdata).
[2023-02-23 23:45:12 (+00:00:32) INFO C++] Catalogue loaded: 1308287 particles with total sample weights 1308287.000 (source=extdata).
[2023-02-23 23:45:12 (+00:00:32) INFO C++] Extents of particle coordinates: {'x': (0.604, 999.396), 'y': (0.63

In the case above, the lines of sight are computed automatically, but one could
supply external data arrays as replacements:

In [9]:
# import numpy as np
results = compute_bispec(
    catalogue_survey, catalogue_rand,
    los_data=np.ones((len(catalogue_survey), 3)),
    los_rand=np.ones((len(catalogue_rand), 3)),
    paramset=parameter_set,
    logger=trv_logger
)

[2023-02-23 23:46:26 (+00:01:46) INFO] Parameter set have been initialised.
[2023-02-23 23:46:26 (+00:01:46) STAT C++] Parameters validated.
[2023-02-23 23:46:26 (+00:01:46) INFO] Binning has been initialised.
[2023-02-23 23:46:26 (+00:01:46) INFO] Lines of sight have been initialised.
[2023-02-23 23:46:26 (+00:01:46) INFO] Catalogues have been aligned.
[2023-02-23 23:46:26 (+00:01:46) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-23 23:46:26 (+00:01:46) INFO C++] Catalogue loaded: 259444 particles with total sample weights 259444.000 (source=extdata).
[2023-02-23 23:46:26 (+00:01:46) INFO C++] Extents of particle coordinates: {'x': (2.438, 998.884), 'y': (0.879, 998.351), 'z': (0.364, 998.843)} (source=extdata).
[2023-02-23 23:46:27 (+00:01:47) INFO C++] Catalogue loaded: 1308287 particles with total sample weights 1308287.000 (source=extdata).
[2023-02-23 23:46:27 (+00:01:47) INFO C++] Extents of particle coordinates: {'x': (0.604, 999.396), 'y': (0.63

You can also override `paramset` (which may be unset as shown in the example
below) by passing the relevant keyword arguments:

In [10]:
# DEMO
# import warnings
warnings.filterwarnings('ignore', message=".*default values are unchanged.*")

results = compute_bispec(
    catalogue_survey, catalogue_rand,
    degrees=(1, 1, 0),
    binning=binning,
    form='full',
    idx_bin=5,
    sampling_params={
        'assignment': 'cic',
        'boxsize': [1000.,]*3,
        'ngrid': [64,]*3
    },
    logger=trv_logger
)

[2023-02-23 23:47:39 (+00:02:59) INFO] Validating parameters... (entering C++)
[2023-02-23 23:47:39 (+00:02:59) INFO] ... validated parameters. (exited C++)
[2023-02-23 23:47:39 (+00:02:59) STAT C++] Parameters validated.
[2023-02-23 23:47:39 (+00:02:59) INFO] Parameter set have been initialised.
[2023-02-23 23:47:39 (+00:02:59) INFO] Binning has been initialised.
[2023-02-23 23:47:40 (+00:03:00) INFO] Lines of sight have been initialised.
[2023-02-23 23:47:40 (+00:03:00) INFO] Catalogues have been aligned.
[2023-02-23 23:47:40 (+00:03:00) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-23 23:47:40 (+00:03:00) INFO C++] Catalogue loaded: 259444 particles with total sample weights 259444.000 (source=extdata).
[2023-02-23 23:47:40 (+00:03:00) INFO C++] Extents of particle coordinates: {'x': (2.438, 998.884), 'y': (0.879, 998.351), 'z': (0.364, 998.843)} (source=extdata).
[2023-02-23 23:47:41 (+00:03:02) INFO C++] Catalogue loaded: 1308287 particles with tota

For other measurement algorithms, the syntax is very similar except for a few
minor differences:

- For two-point clustering statistics, the argument corresponding to `degrees`
  above is `degree` as there is only a single multipole degree. The arguments
  `form` and `idx_bin` do not apply.

- For global plane-parallel measurements, no random catalogue is required.

- For window function measurements, only the random catalogue is required.

For full details, please consult the API reference
({py:mod}`~triumvirate.twopt` and {py:mod}`~triumvirate.threept`).

## Output

The returned measurement results are dictionaries containing the
raw statistic (key with suffix ``_raw``) without shot noise subtraction,
the shot noise (key with suffix ``_shot``), the bin centres for each
coordinate dimension (key with suffix ``bin``), the average/effectuve bin
coordinates (key with suffix ``eff``), and the number of contributing modes
in each bin (key ``'nmodes'``).

In [11]:
# DEMO
from pprint import pprint

pprint(results)

{'bk_raw': array([-6.15401315e+08-3.19482081e-07j,  2.40744325e+08+1.72304943e-07j,
       -2.84005114e+08-9.51266872e-08j, -1.06997676e+08-1.25639021e-07j,
        1.94789754e+07+1.21151913e-08j,  8.29519632e+07+2.01581333e-08j,
       -4.53112276e+06+5.02556083e-08j, -1.06546658e+08-3.58968631e-08j,
       -6.84846274e+07-1.52561668e-08j, -1.14891079e+08+1.61535884e-08j]),
 'bk_shot': array([ -5212454.33669118+8.88525628e-11j,
        -8944908.7000952 -5.25045373e-11j,
       -15387868.92505237+3.95731958e-11j,
       -17045970.3545955 +1.09983928e-11j,
       -21370062.6083337 +3.75538914e-11j,
       -13373351.39063533-6.27183791e-11j,
       -19646302.2498707 +9.65720444e-11j,
       -16373587.0472606 +6.97114100e-11j,
       -14739949.24948343+6.86204187e-11j,
       -12282473.18290978+4.97694435e-11j]),
 'k1bin': array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06]),
 'k1eff': array([0.06040971, 0.06040971, 0.06040971, 0.06040971, 0.06040971,
       0.06040971, 0.0

In each algorithmic function for different measurements, if one sets
``save='.txt'`` or ``save='.npz'``, the results as a dictionary will be
automatically saved to a file in either ``.txt`` or ``.npz`` format.

If the `paramset` argument is set to a
{py:class}`~triumvirate.parameters.ParameterSet` object, the output directory
will be ``paramset['directories']['measurements']``, and the string
``paramset['tags']['output']`` will be appended to the file name before
the extension suffix. An empty output directory string points to the
current working directory. This is demonstrated below:

In [12]:
from triumvirate.twopt import compute_powspec_in_gpp_box

# DEMO
parameter_set.update(tags={'output': '_demo'})

results = compute_powspec_in_gpp_box(
    catalogue_sim,
    degree=0, paramset=parameter_set,
    save='.txt', logger=trv_logger
)

[2023-02-23 23:47:54 (+00:03:14) INFO] Parameter set have been initialised.
[2023-02-23 23:47:54 (+00:03:14) STAT C++] Parameters validated.
[2023-02-23 23:47:54 (+00:03:14) INFO] Binning has been initialised.
[2023-02-23 23:47:54 (+00:03:14) STAT C++] Parameters validated.
[2023-02-23 23:47:54 (+00:03:14) INFO] Catalogue box has been periodised.
[2023-02-23 23:47:54 (+00:03:14) INFO] Inserted missing 'nz' field based on particle count and boxsize.
[2023-02-23 23:47:54 (+00:03:14) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-23 23:47:54 (+00:03:15) INFO C++] Catalogue loaded: 499214 particles with total sample weights 499214.000 (source=extdata).
[2023-02-23 23:47:54 (+00:03:15) INFO C++] Extents of particle coordinates: {'x': (0.000, 1000.000), 'y': (0.000, 999.996), 'z': (0.001, 999.997)} (source=extdata).
[2023-02-23 23:47:54 (+00:03:15) INFO] ... prepared catalogue for clustering algorithm. (exited C++)
[2023-02-23 23:47:55 (+00:03:15) INFO] Normali

Let's have a look at the output measurement file:

In [13]:
# DEMO
with open("pk0_demo.txt", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extdata
# Catalogue size: 499214 particles of total weight 499214.000
# Catalogue particle extents: ([0.000, 1000.000], [0.000, 999.996], [0.001, 999.997])
# Box size: (1000.000, 1000.000, 1000.000)
# Box alignment: centre
# Mesh number: (256, 256, 256)
# Mesh assignment and interlacing: tsc, false
# Normalisation factor: 4.012605716e-03 (particle-based, used), 4.283253172e-04 (mesh-based, alternative)
# [0] k_cen, [1] k_eff, [2] nmodes, [3] Re{pk0_raw}, [4] Im{pk0_raw}, [5] Re{pk0_shot}, [6] Im{pk0_shot}
1.000000000e-02	1.149964290e-02	        56	 3.160453651e+04	 0.000000000e+00	 2.003148950e+03	 0.000000000e+00
2.000000000e-02	2.047114034e-02	       194	 3.881278667e+04	 0.000000000e+00	 2.003148950e+03	 0.000000000e+00
3.000000000e-02	3.052421724e-02	       488	 2.746447398e+04	 0.000000000e+00	 2.003148950e+03	 0.000000000e+00
4.000000000e-02	4.062536317e-02	       812	 2.298579911e+04	 0.000000000e+00	 2.003148950e+03	 0.000000000e+00
5.000000000e-02	5.0333787

We see that a header with summary information about the input parameters and
data have also be included in the saved file.

Similarly, for the ``.npz`` output file, we have

In [15]:
results = compute_powspec_in_gpp_box(
    catalogue_sim,
    degree=0, paramset=parameter_set,
    save='.npz', logger=trv_logger
)

[2023-02-23 23:49:22 (+00:04:42) INFO] Parameter set have been initialised.
[2023-02-23 23:49:22 (+00:04:42) STAT C++] Parameters validated.
[2023-02-23 23:49:22 (+00:04:42) INFO] Binning has been initialised.
[2023-02-23 23:49:22 (+00:04:42) INFO] Catalogue box has been periodised.
[2023-02-23 23:49:22 (+00:04:42) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-23 23:49:23 (+00:04:43) INFO C++] Catalogue loaded: 499214 particles with total sample weights 499214.000 (source=extdata).
[2023-02-23 23:49:23 (+00:04:43) INFO C++] Extents of particle coordinates: {'x': (0.000, 1000.000), 'y': (0.000, 999.996), 'z': (0.001, 999.997)} (source=extdata).
[2023-02-23 23:49:23 (+00:04:43) INFO] ... prepared catalogue for clustering algorithm. (exited C++)
[2023-02-23 23:49:23 (+00:04:43) INFO] Normalisation factors: 4.012606e-03 (used), 4.283253e-04 (alternative).
[2023-02-23 23:49:23 (+00:04:43) INFO] Measuring clustering statistics... (entering C++)
[2023-02-23 23:

In [16]:
# DEMO
with np.load("pk0_demo.npz", allow_pickle=True) as results_file:
    print(results_file['header'])

Catalogue source: extdata
Catalogue size: 499214 particles of total weight 499214.000
Catalogue particle extents: ([0.000, 1000.000], [0.000, 999.996], [0.001, 999.997])
Box size: (1000.000, 1000.000, 1000.000)
Box alignment: centre
Mesh number: (256, 256, 256)
Mesh assignment and interlacing: tsc, false
Normalisation factor: 4.012605716e-03 (particle-based, used), 4.283253172e-04 (mesh-based, alternative)
[0] k_cen, [1] k_eff, [2] nmodes, [3] Re{pk0_raw}, [4] Im{pk0_raw}, [5] Re{pk0_shot}, [6] Im{pk0_shot}
